**Wrangling Data from Laboratory Reports**

Python is a good programming language choice for the medical director/data scientist who needs to parse text from lab reports, paramedical exams, and underwriting notes. While some data sources require sophisticated programming techniques, the parsing of many common sources such as laboratory results is well within the reach of a medical director who is a programming novice.

In [ ]:
print("Hello")

  fgads,f; jkasdlfjasldkj
  jsklJSLKFJ